In [12]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [13]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [14]:
subject = 'Rondônia - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [15]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_RO.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Rondônia - Produção de Cimento (t)', axis=1)
data

,Ano,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),...,Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Rondônia - Consumo de Cimento (t)
0,2008,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,0.693174,5.483335e+08,...,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682,390.788000
1,2009,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,0.693109,5.302764e+08,...,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049,452.325000
2,2010,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,0.693063,5.046425e+08,...,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629,921.425000
3,2011,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,0.693036,4.854622e+08,...,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734,1021.570000
4,2012,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,0.692995,4.867661e+08,...,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622,933.501000
5,2013,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,0.693078,5.225848e+08,...,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194,826.670000
6,2014,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,0.693001,5.676413e+08,...,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147,614.394714
7,2015,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,0.693168,5.179392e+08,...,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726,510.273771
8,2016,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,0.693113,4.524144e+08,...,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474,410.580514
9,2017,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,0.693760,4.213689e+08,...,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241,406.814000


In [16]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,0.751494,7.979888,2.208447e+07,1.637885e+06,11.689351,1.870941e+07,0.698932,0.693174,5.483335e+08,2.546644e+09,...,2.868108,11.760983,0.655238,22.808561,1006.894941,19176.848685,26.497969,21.685776,75.469241,5.070682
1,0.752810,8.008176,2.226473e+07,1.643078e+06,11.684640,1.873529e+07,0.700638,0.693109,5.302764e+08,2.679945e+09,...,2.305885,9.472585,0.659243,19.588422,1006.775008,19127.730770,26.528813,21.707671,75.437690,4.902049
2,0.753706,7.971248,2.285838e+07,1.640521e+06,11.724602,1.891099e+07,0.702254,0.693063,5.046425e+08,2.891930e+09,...,1.958246,9.332485,0.663312,18.596388,1006.795758,14826.071459,26.434620,21.583453,75.296199,4.742629
3,0.754192,7.956614,2.338128e+07,1.618454e+06,11.764179,1.903017e+07,0.703786,0.693036,4.854622e+08,3.091988e+09,...,2.079088,10.985689,0.667449,18.419365,1007.036992,8438.763610,26.447727,21.511515,74.910155,4.560734
4,0.754233,6.546489,2.383098e+07,1.560372e+06,11.757682,1.910530e+07,0.705401,0.692995,4.867661e+08,3.189506e+09,...,2.183282,8.159014,0.672215,19.487867,1007.195877,4928.830653,26.509272,21.598374,75.028438,4.420622
5,0.753919,6.917221,2.422272e+07,1.518707e+06,11.741719,1.916060e+07,0.706351,0.693078,5.225848e+08,3.093872e+09,...,2.403445,7.913561,0.675149,21.002459,1007.408812,3448.198287,26.513099,21.752006,75.695263,4.276194
6,0.753246,7.287260,2.458709e+07,1.494999e+06,11.767208,1.922796e+07,0.707964,0.693001,5.676413e+08,2.779340e+09,...,2.748397,10.350442,0.680412,20.055465,1007.673651,2597.909114,26.561558,21.940898,76.327127,4.207147
7,0.752267,7.654493,2.486993e+07,1.466317e+06,11.788091,1.927573e+07,0.709294,0.693168,5.179392e+08,2.561731e+09,...,2.501529,12.454398,0.684863,18.767358,1007.697489,2136.164001,26.644505,22.123957,76.795244,4.212726
8,0.759242,7.643673,2.507534e+07,1.450033e+06,11.811765,1.931637e+07,0.714910,0.693113,4.524144e+08,2.516859e+09,...,2.073464,13.111908,0.695058,17.848044,1007.584830,1863.004107,26.594749,22.202418,77.390294,4.183474
9,0.758083,7.282097,2.517074e+07,1.454651e+06,11.835195,1.934371e+07,0.714902,0.693760,4.213689e+08,2.497710e+09,...,2.177386,9.479812,0.695427,17.563275,1007.547716,1682.028677,26.577385,22.310278,77.982455,4.075241


In [17]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
12     502.824000
13     501.142000
14            NaN
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [18]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.366933,1.128601,-1.793068,1.343669,-1.602309,-1.897543,-1.597405,0.095187,1.204752,-0.990234,...,2.026352,0.984445,-1.605695,2.457245,-1.212494,1.862951,-0.705394,-0.762192,-0.756764,1.972247
1,-0.774200,1.188885,-1.627168,1.412024,-1.699639,-1.776559,-1.284617,-0.087133,0.820657,-0.546688,...,0.085558,-0.051710,-1.324921,0.144226,-1.582145,1.855564,-0.347758,-0.699954,-0.781667,1.489960
2,-0.370197,1.110188,-1.080790,1.378366,-0.873963,-0.955015,-0.988265,-0.215819,0.275392,0.158672,...,-1.114490,-0.115145,-1.039610,-0.568350,-1.518190,1.208644,-1.439934,-1.053053,-0.893344,1.034020
3,-0.151306,1.079003,-0.599524,1.087914,-0.056217,-0.397801,-0.707210,-0.291249,-0.132596,0.824346,...,-0.697344,0.633403,-0.749553,-0.695505,-0.774675,0.248067,-1.287958,-1.257539,-1.198046,0.513803
4,-0.132584,-1.926102,-0.185632,0.323418,-0.190465,-0.046510,-0.410992,-0.406659,-0.104860,1.148829,...,-0.337667,-0.646476,-0.415389,0.071997,-0.284967,-0.279787,-0.574331,-1.010638,-1.104686,0.113084
5,-0.274035,-1.136039,0.174914,-0.224995,-0.520296,0.212086,-0.236740,-0.174695,0.657046,0.830615,...,0.422337,-0.757614,-0.209634,1.159926,0.371328,-0.502457,-0.529965,-0.573928,-0.578367,-0.299977
6,-0.577490,-0.347451,0.510268,-0.537052,0.006355,0.527031,0.059020,-0.389665,1.615453,-0.215960,...,1.613108,0.345772,0.159340,0.479702,1.187601,-0.630330,0.031926,-0.036991,-0.079642,-0.497452
7,-1.018893,0.435155,0.770583,-0.914581,0.437846,0.750387,0.303008,0.076848,0.558230,-0.940033,...,0.760923,1.298413,0.471418,-0.445543,1.261073,-0.699772,0.993697,0.483365,0.289839,-0.481495
8,2.124681,0.412096,0.959640,-1.128914,0.927003,0.940414,1.333009,-0.076826,-0.835563,-1.089341,...,-0.716758,1.596124,1.186272,-1.105884,0.913840,-0.740852,0.416771,0.706397,0.759507,-0.565156
9,1.602027,-0.358454,1.047447,-1.068126,1.411104,1.068244,1.331508,1.732167,-1.495939,-1.153056,...,-0.358019,-0.048438,1.212138,-1.310433,0.799449,-0.768068,0.215438,1.012997,1.226895,-0.874702


In [19]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      452.325000
1      921.425000
2     1021.570000
3      933.501000
4      826.670000
5      614.394714
6      510.273771
7      410.580514
8      406.814000
9      376.138000
10     399.441000
11     488.194000
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [20]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-2.881361,1.108405,2.635823,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


In [21]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    502.824
Name: Rondônia - Consumo de Cimento (t), dtype: float64

In [22]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [23]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [25]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3688570636, 2553090371, 2329880139, 3598513935, 4233264563, 2000869251, 3138027667, 3751372990, 2470068592, 2210172929, 3625234675, 306583789, 2729072690, 713090065, 2154881929, 2628897713, 2295390926, 1681448852, 1205878731, 1011981994, 3585287955, 1771796125, 616289332, 728234088, 1668828547, 1658408624, 4242117832, 1309290196, 625123417, 3244289762, 2555595662, 3308473641, 1382905717, 3113027440, 2807765654, 1007762266, 631713734, 1207981525, 3095018446, 1409644251, 3033944272, 2552597042, 4125228713, 150032035, 1264967951, 540325999, 3646787287, 63479322, 3368682185, 119496591, 2402717030, 675109096, 382313976, 1330456685, 3061386654, 1162988877, 2309128190, 709678698, 514880112, 556138627, 2648737173, 2138083868, 2418251064, 3598795889, 119378647, 3763381675, 2769310867, 3053158301, 1751853617, 1796534593, 1607512775, 3515312908, 8059762, 3306393094, 1173247319, 860948130, 986486963, 3617947159, 3355644918, 123526220, 3121453086, 2395344194, 543760810, 655219887, 871499633, 27760

val_loss: 1081.4580078125


Step: 85 ___________________________________________
val_loss: 2750.155517578125


Step: 86 ___________________________________________
val_loss: 3168.572021484375


Step: 87 ___________________________________________
val_loss: 3168.582275390625


Step: 88 ___________________________________________
val_loss: 4121.39453125


Step: 89 ___________________________________________
val_loss: 1302.080322265625


Step: 90 ___________________________________________
val_loss: 339.7557373046875


Step: 91 ___________________________________________
val_loss: 3167.3798828125


Step: 92 ___________________________________________
val_loss: 1920.6771240234375


Step: 93 ___________________________________________
val_loss: 717.3873901367188


Step: 94 ___________________________________________
val_loss: 0.007756981998682022


Step: 95 ___________________________________________


2023-09-15 19:07:59.806048: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled
2023-09-15 19:07:59.835682: W tensorflow/core/data/root_dataset.cc:286] Optimization loop failed: CANCELLED: Operation was cancelled


val_loss: 3168.572021484375


Step: 96 ___________________________________________
val_loss: 82.54261779785156


Step: 97 ___________________________________________
val_loss: 2847.844970703125


Step: 98 ___________________________________________
val_loss: 3168.572021484375


Step: 99 ___________________________________________
val_loss: 1139.7979736328125


final_seed: 556138627


In [26]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 480ms/step - loss: 430469.3438 - val_loss: 246006.7656
Epoch 2/10000
1/1 [==============================] - 0s 19ms/step - loss: 425086.5938 - val_loss: 206579.3438
Epoch 3/10000
1/1 [==============================] - 0s 15ms/step - loss: 424365.3438 - val_loss: 273956.5312
Epoch 4/10000
1/1 [==============================] - 0s 15ms/step - loss: 406870.3750 - val_loss: 457348.5000
Epoch 5/10000
1/1 [==============================] - 0s 15ms/step - loss: 320940.7812 - val_loss: 504477.7812
Epoch 6/10000
1/1 [==============================] - 0s 15ms/step - loss: 346581.2188 - val_loss: 289030.4062
Epoch 7/10000
1/1 [==============================] - 0s 15ms/step - loss: 350073.3438 - val_loss: 435022.5312
Epoch 8/10000
1/1 [==============================] - 0s 15ms/step - loss: 329235.1875 - val_loss: 308877.5938
Epoch 9/10000
1/1 [==============================] - 0s 14ms/step - loss: 340204.4375 - val_loss: 417771.5938
Epoch 10/

1/1 [==============================] - 0s 15ms/step - loss: 75598.2188 - val_loss: 61683.1523
Epoch 76/10000
1/1 [==============================] - 0s 14ms/step - loss: 74403.5312 - val_loss: 51875.4570
Epoch 77/10000
1/1 [==============================] - 0s 14ms/step - loss: 73368.2188 - val_loss: 50168.5781
Epoch 78/10000
1/1 [==============================] - 0s 15ms/step - loss: 73567.5312 - val_loss: 48993.9062
Epoch 79/10000
1/1 [==============================] - 0s 15ms/step - loss: 70765.9141 - val_loss: 44589.7734
Epoch 80/10000
1/1 [==============================] - 0s 15ms/step - loss: 67964.1328 - val_loss: 46135.0234
Epoch 81/10000
1/1 [==============================] - 0s 14ms/step - loss: 69338.0234 - val_loss: 41013.1328
Epoch 82/10000
1/1 [==============================] - 0s 19ms/step - loss: 69724.3203 - val_loss: 40513.6875
Epoch 83/10000
1/1 [==============================] - 0s 15ms/step - loss: 68879.1094 - val_loss: 39121.4180
Epoch 84/10000
1/1 [==============

Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 38788.9023 - val_loss: 24488.8496
Epoch 151/10000
1/1 [==============================] - 0s 14ms/step - loss: 38420.5898 - val_loss: 23472.2363
Epoch 152/10000
1/1 [==============================] - 0s 14ms/step - loss: 41065.5820 - val_loss: 25446.5781
Epoch 153/10000
1/1 [==============================] - 0s 14ms/step - loss: 37980.2461 - val_loss: 23428.7363
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 37720.8672 - val_loss: 22702.8867
Epoch 155/10000
1/1 [==============================] - 0s 15ms/step - loss: 37491.9180 - val_loss: 22078.7148
Epoch 156/10000
1/1 [==============================] - 0s 14ms/step - loss: 37768.7305 - val_loss: 16138.2490
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 37478.8477 - val_loss: 19195.7363
Epoch 158/10000
1/1 [==============================] - 0s 15ms/step - loss: 37095.5820 - val_loss: 19178.6230
Epoch 159/

1/1 [==============================] - 0s 14ms/step - loss: 26981.3535 - val_loss: 8960.2275
Epoch 225/10000
1/1 [==============================] - 0s 15ms/step - loss: 26837.4941 - val_loss: 8421.9775
Epoch 226/10000
1/1 [==============================] - 0s 15ms/step - loss: 26703.8496 - val_loss: 7980.6318
Epoch 227/10000
1/1 [==============================] - 0s 15ms/step - loss: 26337.3379 - val_loss: 7999.4487
Epoch 228/10000
1/1 [==============================] - 0s 15ms/step - loss: 27150.0234 - val_loss: 6676.8818
Epoch 229/10000
1/1 [==============================] - 0s 15ms/step - loss: 26149.0918 - val_loss: 7162.7852
Epoch 230/10000
1/1 [==============================] - 0s 14ms/step - loss: 26881.9629 - val_loss: 7829.9761
Epoch 231/10000
1/1 [==============================] - 0s 14ms/step - loss: 25942.8516 - val_loss: 7147.7871
Epoch 232/10000
1/1 [==============================] - 0s 14ms/step - loss: 25842.7344 - val_loss: 6946.6230
Epoch 233/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 22368.1191 - val_loss: 4079.8635
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 22039.4844 - val_loss: 3524.9521
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 22907.5547 - val_loss: 3284.3728
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 21931.5254 - val_loss: 3140.3936
Epoch 303/10000
1/1 [==============================] - 0s 15ms/step - loss: 21864.7188 - val_loss: 3014.3042
Epoch 304/10000
1/1 [==============================] - 0s 14ms/step - loss: 22200.7891 - val_loss: 3383.4417
Epoch 305/10000
1/1 [==============================] - 0s 14ms/step - loss: 21913.0254 - val_loss: 2989.0706
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 22341.3379 - val_loss: 2253.3835
Epoch 307/10000
1/1 [==============================] - 0s 14ms/step - loss: 22040.8438 - val_loss: 2948.1738
Epoch 308/10000
1/1 [==============

...

1/1 [==============================] - 0s 14ms/step - loss: 18073.3105 - val_loss: 703.6562
Epoch 1510/10000
1/1 [==============================] - 0s 15ms/step - loss: 17413.2988 - val_loss: 750.3801
Epoch 1511/10000
1/1 [==============================] - 0s 14ms/step - loss: 17286.4219 - val_loss: 871.0496
Epoch 1512/10000
1/1 [==============================] - 0s 14ms/step - loss: 17244.7441 - val_loss: 1288.5883
Epoch 1513/10000
1/1 [==============================] - 0s 14ms/step - loss: 17281.9355 - val_loss: 1162.4795
Epoch 1514/10000
1/1 [==============================] - 0s 14ms/step - loss: 17274.5664 - val_loss: 1147.5073
Epoch 1515/10000
1/1 [==============================] - 0s 14ms/step - loss: 17419.3496 - val_loss: 966.8403
Epoch 1516/10000
1/1 [==============================] - 0s 15ms/step - loss: 17274.3027 - val_loss: 1042.1284
Epoch 1517/10000
1/1 [==============================] - 0s 14ms/step - loss: 18107.7363 - val_loss: 1239.5630
Epoch 1518/10000
1/1 [=========

1/1 [==============================] - 0s 15ms/step - loss: 18910.5723 - val_loss: 553.9194
Epoch 1585/10000
1/1 [==============================] - 0s 14ms/step - loss: 18185.2266 - val_loss: 841.5169
Epoch 1586/10000
1/1 [==============================] - 0s 14ms/step - loss: 17389.5020 - val_loss: 456.8450
Epoch 1587/10000
1/1 [==============================] - 0s 14ms/step - loss: 17320.8398 - val_loss: 310.7966
Epoch 1588/10000
1/1 [==============================] - 0s 14ms/step - loss: 17237.5410 - val_loss: 529.4915
Epoch 1589/10000
1/1 [==============================] - 0s 15ms/step - loss: 17426.3516 - val_loss: 799.1923
Epoch 1590/10000
1/1 [==============================] - 0s 14ms/step - loss: 17378.2031 - val_loss: 430.2092
Epoch 1591/10000
1/1 [==============================] - 0s 15ms/step - loss: 17314.0312 - val_loss: 296.9932
Epoch 1592/10000
1/1 [==============================] - 0s 14ms/step - loss: 17486.3027 - val_loss: 651.6016
Epoch 1593/10000
1/1 [==============

1/1 [==============================] - 0s 15ms/step - loss: 17184.4395 - val_loss: 857.6097
Epoch 1660/10000
1/1 [==============================] - 0s 15ms/step - loss: 17566.1484 - val_loss: 609.0146
Epoch 1661/10000
1/1 [==============================] - 0s 14ms/step - loss: 18299.5781 - val_loss: 759.3298
Epoch 1662/10000
1/1 [==============================] - 0s 14ms/step - loss: 17173.8398 - val_loss: 827.1048
Epoch 1663/10000
1/1 [==============================] - 0s 14ms/step - loss: 17170.7012 - val_loss: 871.8172
Epoch 1664/10000
1/1 [==============================] - 0s 14ms/step - loss: 17989.0469 - val_loss: 746.4145
Epoch 1665/10000
1/1 [==============================] - 0s 18ms/step - loss: 17351.4766 - val_loss: 442.3789
Epoch 1666/10000
1/1 [==============================] - 0s 19ms/step - loss: 18280.7109 - val_loss: 89.3547
Epoch 1667/10000
1/1 [==============================] - 0s 15ms/step - loss: 18472.7285 - val_loss: 691.2352
Epoch 1668/10000
1/1 [===============

1/1 [==============================] - 0s 15ms/step - loss: 17337.5703 - val_loss: 1754.1240
Epoch 1735/10000
1/1 [==============================] - 0s 15ms/step - loss: 17275.6777 - val_loss: 1480.5446
Epoch 1736/10000
1/1 [==============================] - 0s 14ms/step - loss: 18334.9941 - val_loss: 2713.4314
Epoch 1737/10000
1/1 [==============================] - 0s 14ms/step - loss: 17189.9199 - val_loss: 2664.2080
Epoch 1738/10000
1/1 [==============================] - 0s 14ms/step - loss: 18275.0059 - val_loss: 2736.4158
Epoch 1739/10000
1/1 [==============================] - 0s 14ms/step - loss: 17374.9473 - val_loss: 3107.5044
Epoch 1740/10000
1/1 [==============================] - 0s 14ms/step - loss: 18037.6367 - val_loss: 3944.6282
Epoch 1741/10000
1/1 [==============================] - 0s 14ms/step - loss: 17514.2090 - val_loss: 2658.6128
Epoch 1742/10000
1/1 [==============================] - 0s 14ms/step - loss: 18278.0410 - val_loss: 2895.2827
Epoch 1743/10000
1/1 [=====

1/1 [==============================] - 0s 14ms/step - loss: 17220.0215 - val_loss: 224.3294
Epoch 1810/10000
1/1 [==============================] - 0s 14ms/step - loss: 17431.9355 - val_loss: 380.5678
Epoch 1811/10000
1/1 [==============================] - 0s 15ms/step - loss: 17228.3984 - val_loss: 583.2517
Epoch 1812/10000
1/1 [==============================] - 0s 14ms/step - loss: 17554.0410 - val_loss: 717.0016
Epoch 1813/10000
1/1 [==============================] - 0s 20ms/step - loss: 18159.1699 - val_loss: 264.4247
Epoch 1814/10000
1/1 [==============================] - 0s 15ms/step - loss: 18024.7012 - val_loss: 106.6676
Epoch 1815/10000
1/1 [==============================] - 0s 14ms/step - loss: 17692.1328 - val_loss: 347.4774
Epoch 1816/10000
1/1 [==============================] - 0s 14ms/step - loss: 17413.3262 - val_loss: 456.7041
Epoch 1817/10000
1/1 [==============================] - 0s 14ms/step - loss: 18079.4688 - val_loss: 148.6088
Epoch 1818/10000
1/1 [==============

In [27]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 49ms/step
[502.824](test_target) - [[440.06726]](prediction) = 62.75673925781251


In [28]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [29]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
0,-1.294335,-0.415281,-0.961963,-1.230898,-0.570897,-0.846639,-1.235610,1.288777,1.148205,-1.122043,...,1.308949,1.412346,-1.221460,1.377871,1.395565,0.719114,0.276338,0.495192,0.906319,1.236047
1,0.153687,1.378391,-0.416778,1.218498,-0.835068,-0.557701,0.022028,-0.140116,0.140893,-0.184474,...,-0.190804,-0.643250,-0.006544,-0.413067,-0.896020,0.695031,1.062967,0.899613,0.487021,-0.022926
2,1.140648,-0.963110,1.378740,0.012400,1.405965,1.404340,1.213582,-1.148661,-1.289098,1.306518,...,-1.118144,-0.769096,1.228004,-0.964804,-0.499545,-1.414145,-1.339305,-1.394805,-1.393340,-1.213121


train_target:


,Rondônia - Consumo de Cimento (t)
0,452.325
1,921.425
2,1021.570


test_input:


...

,Rondônia - IDH Longevidade,Rondônia - Desemprego,Rondônia - PIB - Estadual,Rondônia - PIB - Construção Civil,Rondônia - PIB - Per Capita,Rondônia - PIB - Preços de Mercado,Rondônia - IDH,Rondônia - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,NFSP - Porcentagem do PIB (%),Taxa Selic (%),Rondônia - IDH Educacao,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s)
12,-0.970117,-1.191413,0.552069,-0.469187,0.356059,0.388444,0.645784,-2.881361,1.108405,2.635823,...,-1.482779,-2.260447,0.964061,-2.07127,-0.395184,-0.716176,2.608857,1.699552,1.207817,-1.029715


test_target:


,Rondônia - Consumo de Cimento (t)
12,502.824


1/1 [==============================] - 0s 35ms/step
Error: 62.75673925781251




[1021.5699462890625,
 972.128173828125,
 820.0667114257812,
 602.076904296875,
 392.9931945800781,
 405.50311279296875,
 362.7301025390625,
 378.1610107421875,
 416.15667724609375,
 440.0672607421875]

In [30]:
display(targets)
display(predictions)

[933.501,
 826.67,
 614.3947142857144,
 510.2737714285714,
 410.5805142857144,
 406.814,
 376.138,
 399.441,
 488.194,
 502.824]

[1021.5699462890625,
 972.128173828125,
 820.0667114257812,
 602.076904296875,
 392.9931945800781,
 405.50311279296875,
 362.7301025390625,
 378.1610107421875,
 416.15667724609375,
 440.0672607421875]

In [31]:
mae = mean_absolute_error(predictions, targets)
mae

71.93824057686943

In [32]:
porcentage = mae/np.mean(targets)
porcentage

0.13154226301172853